<a href="https://colab.research.google.com/github/Ubaid-Manzoor/N-Layer-Neural-Network-Using-TensorFlow-With-Mnist-Dataset/blob/master/N_Layer_CNN(Mnist_dataset).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from tensorflow.python.framework import ops
import numpy as np
import tensorflow as tf
import math
import matplotlib.pyplot as plt

np.random.seed(1)

In [0]:
mnist = tf.keras.datasets.mnist

(X_train , y_train) , (X_test , y_test) = mnist.load_data()

In [0]:
y_train = tf.one_hot(y_train , len(np.unique(y_train)))
y_test = tf.one_hot(y_test , len(np.unique(y_test)))

In [0]:
print(X_train.shape)
print(X_test.shape)

(60000, 28, 28)
(10000, 28, 28)


In [0]:
with tf.Session() as sess:
  y_train = sess.run(y_train)
  y_test = sess.run(y_test)
print(y_train.shape)
print(y_test.shape)

(60000, 10)
(10000, 10)


In [0]:
def create_placeholder(n_H , n_W , n_C , n_y):
  
  X = tf.placeholder(shape=[None , n_H, n_W, n_C] , dtype=tf.float32 , name='X')
  Y = tf.placeholder(shape=[None, n_y] , dtype=tf.float32 , name="Y")
  
  return X,Y

In [0]:
#Create a Parameters dict to Hold all Weights

parameters = {}
Weight_shapes = [[4,4,3,5] , [3,3,5,10] , [2,2,10,15]]# length of Weight_Shape depends on number of layers
for i,shape in enumerate(Weight_shapes):
  l=i+1
  parameters["W"+str(l)] = shape
print(parameters)

{'W1': [4, 4, 3, 5], 'W2': [3, 3, 5, 10], 'W3': [2, 2, 10, 15]}


In [0]:
def initialize_parameters(parameters):
  tf.set_random_seed(1)
  L = len(parameters)+1
  for l in range(1,L):
    parameters["W" + str(l)] = tf.get_variable("W"+str(l) ,parameters["W"+str(l)] 
                                               , initializer=tf.contrib.layers.xavier_initializer(seed = 0))
  print(parameters)

In [0]:
initialize_parameters(parameters)

{'W1': <tf.Variable 'W1:0' shape=(4, 4, 3, 5) dtype=float32_ref>, 'W2': <tf.Variable 'W2:0' shape=(3, 3, 5, 10) dtype=float32_ref>, 'W3': <tf.Variable 'W3:0' shape=(2, 2, 10, 15) dtype=float32_ref>}
